In [ ]:
import numpy as np
import pandas as pd
import polars as pl
from catboost import CatBoostRegressor, Pool
from itertools import combinations

import warnings
warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning,
    message=".*DataFrameGroupBy.apply operated on the grouping columns.*",
)

pd.set_option("compute.use_numexpr", False)
pd.set_option("compute.use_bottleneck", False)

############################### Helper Functions ###############################


# function to calculate direction between two frames
# direction in degrees and on the coordinate system used by BDB (0 deg is vertical, clockwise)
# inputs: x_diff, y_diff between current and previous frame
def get_dir(x_diff, y_diff):
    angle = (90 - np.degrees(np.arctan2(y_diff, x_diff))) % 360
    return angle


# function to calculate distance traveled between two frames
# inputs: distance travelled in x, y coordinates
def get_dist(x_diff, y_diff):
    return np.sqrt(x_diff**2 + y_diff**2)


# function that returns the minimum positive or negative direction between two direction differences
def min_pos_neg_dir(dir_diff):
    pos_diff = dir_diff % 360
    neg_diff = (-dir_diff) % 360
    return np.where(pos_diff <= neg_diff, pos_diff, -neg_diff)
#for eg, dirA = 1 and dirB = 359 has -2 direction difference, not 358
#therefore, max possible direction difference is either 180 or -180 deg



############################################### Feature Derivation Functions ############################################### 

# function that takes in train df and estimates dir, s, and a
# df needs to have columns: player_to_predict, game_player_play_id, x, y
# x,y here can be either the true values or the predicted values
def est_kinematics(df):
    df = df.copy()
    
    #sort by player and frame if needed
    df = df.sort_values(['game_player_play_id', "frame_id"])
    
    #group by player
    df_grouped = df.groupby("game_player_play_id")
    
    # Compute speed, acceleration, direction
    df["x_lag"] = df_grouped["x"].shift(1)
    df["y_lag"] = df_grouped["y"].shift(1)
    
    x_diff = df["x"] - df["x_lag"]
    y_diff = df["y"] - df["y_lag"]

    df["est_speed"] = get_dist(x_diff, y_diff)/0.1
    df["est_acc"] = (df["est_speed"] - df_grouped["est_speed"].shift(1))/0.1
    df["est_dir"] = get_dir(x_diff, y_diff)
    
    # remove 0 speeds since were logging later
    df.loc[(df["est_speed"] == 0) & (df["throw"] == "post"), "est_speed"] = 0.01
    df["est_speed"] = np.where(df["throw"] == "post", df["est_speed"], df["s"])
    df["est_dir"] = np.where(df["throw"] == "post", df["est_dir"], df["dir"])

    #max frame id for this player on this play
    df["max_frame_id"] = df_grouped["frame_id"].transform("max")
    df["time_until_play_complete"] = (df["max_frame_id"] - df["frame_id"])*0.1    

    #time elapsed post throw
    df["rownum_throw_group"] = (df.groupby(["game_player_play_id", "throw"]).cumcount() + 1)
    df["time_elapsed_post_throw"] = np.where(df["throw"] == "pre", np.nan, df["rownum_throw_group"] * 0.1)
    
    # Drop temporary columns
    df = df.drop(columns=["x_lag", "y_lag", "rownum_throw_group"])

   
    return df


# function that takes in training df and gets the previous and future change in dir, s, a
# the prev change can be used as a feature
# the future change is the response
# the input df needs to have kinematics (dir, s, a) for this function to work
def change_in_kinematics(df):
    df = df.copy()

    df["log_est_speed"] = np.log(df["est_speed"])
    
    df_grouped = df.groupby("game_player_play_id")
    
    df["prev_dir_diff"] = min_pos_neg_dir(df["est_dir"] - df_grouped["est_dir"].shift(1))
    df["prev_a_diff"] = df["est_acc"] - df_grouped["est_acc"].shift(1)

    df["prev_log_s_diff"] = df["log_est_speed"] - df_grouped["log_est_speed"].shift(1)
    
    return df


# function that takes in df and calculates all our derived features
def derived_features(df):
    df = df.copy()

    #time elapsed and time until ball land
    df["time_elapsed"] = df["frame_id"]*0.1
    
    # Direction to ball landing point
    df["curr_ball_land_dir"] = get_dir(df["ball_land_x"] - df["x"], df["ball_land_y"] - df["y"])
    df["ball_land_dir_diff"] = min_pos_neg_dir(df["est_dir"] - df["curr_ball_land_dir"])
    df["dist_ball_land"] = get_dist(df["ball_land_x"] - df["x"], df["ball_land_y"] - df["y"])
    
    # Distance to closest out-of-bounds point
    x, y = df["x"], df["y"]
    
    out_left = x - 0
    out_right = 120 - x
    out_top = 53.3 - y
    out_bottom = y - 0
    # Minimum distance to any boundary
    df["out_bounds_dist"] = pd.concat([out_left, out_right, out_top, out_bottom], axis=1).min(axis=1)
    
    # Direction to closest boundary (0, 90, 180, 270)
    df["out_bounds_dir"] = np.select(
        condlist=[
            df["out_bounds_dist"] == out_left,
            df["out_bounds_dist"] == out_right,
            df["out_bounds_dist"] == out_top,
            df["out_bounds_dist"] == out_bottom
        ],
        choicelist=[270, 90, 0, 180],
        default=np.nan
    )
    
    # Direction difference
    df["out_bounds_dir_diff"] = min_pos_neg_dir(df["est_dir"] - df["out_bounds_dir"])
    
    # Drop temporary columns
    df = df.drop(columns=["curr_ball_land_dir", "out_bounds_dir"])
    
    return df




# takes in a dataframe for all players in the same frame
# outputs the min distance and direction to the closest other player
# input df must have: game_player_play_id, player_side, est_dir, x, y for each player in the frame
def get_closest_player_min_dist_dir(df):
    df = df.reset_index(drop=True)

    # If only one player: return NA fields
    if len(df) == 1:
        return pd.DataFrame({
            "game_player_play_id": [df.iloc[0]["game_player_play_id"]],
            "closest_teammate": [np.nan],
            "closest_teammate_dist": [np.nan],
            "closest_teammate_dir_diff_position": [np.nan],
            "closest_teammate_dir_diff_heading": [np.nan],
            "closest_teammate_speed": [np.nan],
            "closest_teammate_acc": [np.nan],
            "closest_opponent": [np.nan],
            "closest_opponent_dist": [np.nan],
            "closest_opponent_dir_diff_position": [np.nan],
            "closest_opponent_dir_diff_heading": [np.nan],
            "closest_opponent_speed": [np.nan],
            "closest_opponent_acc": [np.nan]
        })

    #pairwise features
    rows = []
    for i, j in combinations(range(len(df)), 2):

        dx_ij = df.loc[j, "x"] - df.loc[i, "x"]
        dy_ij = df.loc[j, "y"] - df.loc[i, "y"]

        dx_ji = -dx_ij
        dy_ji = -dy_ij

        rows.append({
            "id1": df.loc[i, "game_player_play_id"],
            "id2": df.loc[j, "game_player_play_id"],
            "distance": get_dist(dx_ij, dy_ij),

            "player1_dir": df.loc[i, "est_dir"],
            "player2_dir": df.loc[j, "est_dir"],

            "dir12": get_dir(dx_ij, dy_ij),
            "dir21": get_dir(dx_ji, dy_ji),

            "side1": df.loc[i, "player_side"],
            "side2": df.loc[j, "player_side"],

            "player1_speed": df.loc[i, "est_speed"],
            "player1_acc": df.loc[i, "est_acc"],
            "player2_speed": df.loc[j, "est_speed"],
            "player2_acc": df.loc[j, "est_acc"]
        })

    pair_df = pd.DataFrame(rows)

    #player1 long
    p1 = pd.DataFrame({
        "player1": pair_df["id1"],
        "player2": pair_df["id2"],
        "player1_side": pair_df["side1"],
        "player2_side": pair_df["side2"],
        "player1_dir": pair_df["player1_dir"],
        "player2_dir": pair_df["player2_dir"],
        "player2_speed": pair_df["player2_speed"],
        "player2_acc": pair_df["player2_acc"],
        "dir_to_other": pair_df["dir12"],
        "distance": pair_df["distance"]
    })

    #player2 long
    p2 = pd.DataFrame({
        "player1": pair_df["id2"],
        "player2": pair_df["id1"],
        "player1_side": pair_df["side2"],
        "player2_side": pair_df["side1"],
        "player1_dir": pair_df["player2_dir"],
        "player2_dir": pair_df["player1_dir"],
        "player2_speed": pair_df["player1_speed"],
        "player2_acc": pair_df["player1_acc"],
        "dir_to_other": pair_df["dir21"],
        "distance": pair_df["distance"]
    })

    long_df = pd.concat([p1, p2], ignore_index=True)

    #min distances between teammates and opponents
    min_dist = (long_df.sort_values(["player1", "player2_side", "distance"]).groupby(["player1", "player2_side"], as_index=False).first())

    #direction diffs
    min_dist["dir_diff_to_player2"] = min_pos_neg_dir(min_dist["dir_to_other"] - min_dist["player1_dir"])
    min_dist["dir_diff_player12"] = min_pos_neg_dir(min_dist["player2_dir"] - min_dist["player1_dir"])

    #teammates
    teammate = (
        min_dist[min_dist["player1_side"] == min_dist["player2_side"]]
        .rename(columns={
            "player1": "game_player_play_id",
            "player2": "closest_teammate",
            "distance": "closest_teammate_dist",
            "dir_diff_to_player2": "closest_teammate_dir_diff_position",
            "dir_diff_player12": "closest_teammate_dir_diff_heading",
            "player2_speed": "closest_teammate_speed",
            "player2_acc": "closest_teammate_acc"
        })[
            [
                "game_player_play_id",
                "closest_teammate",
                "closest_teammate_dist",
                "closest_teammate_dir_diff_position",
                "closest_teammate_dir_diff_heading",
                "closest_teammate_speed",
                "closest_teammate_acc"
            ]
        ]
    )

    #opponents 
    opponent = (
        min_dist[min_dist["player1_side"] != min_dist["player2_side"]]
        .rename(columns={
            "player1": "game_player_play_id",
            "player2": "closest_opponent",
            "distance": "closest_opponent_dist",
            "dir_diff_to_player2": "closest_opponent_dir_diff_position",
            "dir_diff_player12": "closest_opponent_dir_diff_heading",
            "player2_speed": "closest_opponent_speed",
            "player2_acc": "closest_opponent_acc"
        })[
            [
                "game_player_play_id",
                "closest_opponent",
                "closest_opponent_dist",
                "closest_opponent_dir_diff_position",
                "closest_opponent_dir_diff_heading",
                "closest_opponent_speed",
                "closest_opponent_acc"
            ]
        ]
    )

    #join teammates and opponents together
    out = pd.merge(teammate, opponent, on="game_player_play_id", how="outer")

    return out

In [16]:
"""
The evaluation API requires that you set up a server which will respond to inference requests.
We have already defined the server; you just need write the predict function.
When we evaluate your submission on the hidden test set the client defined in `nfl_gateway` will run in a different container
with direct access to the hidden test set and hand off the data timestep by timestep.
Your code will always have access to the published copies of the copmetition files.
"""

import os

import pandas as pd
import polars as pl

import kaggle_evaluation.nfl_inference_server

def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    #data preprocessing
    test = test.to_pandas()
    test_input = test_input.to_pandas()
    
    test_input_pro = test_input.copy()
    test_input_pro["player_to_predict"] = test_input_pro["player_to_predict"].astype(bool) #make bool
    test_input_pro["throw"] = "pre" #this is pre throw data
    test_input_pro["game_play_id"] = (test_input_pro.groupby(["game_id", "play_id"]).ngroup()) #add game_play_ids
    test_input_pro["game_player_play_id"] = (test_input_pro.groupby(["game_id", "nfl_id", "play_id"]).ngroup()) #add game_player_play_ids
    
    #prop_play_complete = frame_id / (max(frame_id) + num_frames_output)
    test_input_pro["max_frame_pre_throw"] = (test_input_pro.groupby(["game_play_id"])["frame_id"].transform("max"))
    test_input_pro["prop_play_complete"] = (test_input_pro["frame_id"] / (test_input_pro["max_frame_pre_throw"] + test_input_pro["num_frames_output"]))
    
    #filter only player_to_predict
    test_input_pro = test_input_pro[test_input_pro["player_to_predict"]].reset_index(drop=True)
    
    #convert height to inches
    feet_inches = test_input_pro["player_height"].str.split("-", n=1, expand=True)
    test_input_pro["player_height"] = (feet_inches[0].astype(float) * 12 + feet_inches[1].astype(float))
    
    #df for predictions
    data_mod_test_pred = (test_input_pro
                          .sort_values(["game_player_play_id", "frame_id"]) #arrange correct order
                          .groupby("game_player_play_id")    
                          .tail(4) #need last 4 columns pre throw for lag stuff                                               
                          .sort_values("game_play_id") 
                          .reset_index(drop=True))
    
    #derive features
    data_mod_test_pred = est_kinematics(data_mod_test_pred)
    data_mod_test_pred = change_in_kinematics(data_mod_test_pred)
    data_mod_test_pred = derived_features(data_mod_test_pred)
    
    #keep only final pre throw frame
    data_mod_test_pred = (data_mod_test_pred
                          .groupby("game_player_play_id")
                          .tail(1) 
                          .reset_index(drop=True))
    
    #closest features - group by play
    #run on each play - each play here is the final frame pre throw
    close_features = (data_mod_test_pred.copy()
                      .groupby(["game_play_id"])
                      .apply(get_closest_player_min_dist_dir)
                      .reset_index(drop=True))
    #join close features
    data_mod_test_pred = data_mod_test_pred.merge(close_features, on="game_player_play_id", how="outer")
    
                          #arrange back in correct order
    data_mod_test_pred = data_mod_test_pred.sort_values(["game_play_id", "game_player_play_id", "frame_id"])
    
    #dist ball land is out of bounds
    data_mod_test_pred["ball_land_dist_out"] = pd.concat([
        data_mod_test_pred["ball_land_x"],
        120 - data_mod_test_pred["ball_land_x"],
        data_mod_test_pred["ball_land_y"],
        53.3 - data_mod_test_pred["ball_land_y"]
    ], axis=1).min(axis=1)
                                                   
    
    #plays we need to predict on
    test_plays = sorted(data_mod_test_pred["game_play_id"].unique())
    
    #load catboost dir, s, a models
    dir_o = CatBoostRegressor().load_model("/kaggle/input/catboost-bdb/other/default/11/exp_final_models/offense/dir.cbm")
    dir_d = CatBoostRegressor().load_model("/kaggle/input/catboost-bdb/other/default/11/exp_final_models/defense/dir.cbm")
    speed_o = CatBoostRegressor().load_model("/kaggle/input/catboost-bdb/other/default/11/exp_final_models/offense/speed.cbm")
    speed_d = CatBoostRegressor().load_model("/kaggle/input/catboost-bdb/other/default/11/exp_final_models/defense/speed.cbm")
    acc_o = CatBoostRegressor().load_model("/kaggle/input/catboost-bdb/other/default/11/exp_final_models/offense/acc.cbm")
    acc_d = CatBoostRegressor().load_model("/kaggle/input/catboost-bdb/other/default/11/exp_final_models/defense/acc.cbm")
    
    #features for each model
    dir_o_features = dir_o.feature_names_
    dir_d_features = dir_d.feature_names_
    speed_o_features = speed_o.feature_names_
    speed_d_features = speed_d.feature_names_
    acc_o_features = acc_o.feature_names_
    acc_d_features = acc_d.feature_names_
    categorical_features = ["throw", "play_direction", "player_position"]
    
    #now predict through all num_frames_output in each play for each player
    
    #' flow of this:
    #'  -loop through all the plays
    #'    -loop through all the frames in a num_frames_output
    #'    -for each player: predict next x,y and derive new features
    #'      -loop through the players in the play post throw
    #'      -predict next dir, s, a
    
    final_predictions = pd.DataFrame(columns=["game_play_id", "game_player_play_id", "frame_id", "pred_x", "pred_y"])
    
    
    #loop through plays
    for play in test_plays:
        if play % 50 == 0: print(play) #progress
        
        #info for this play
        curr_play_info = data_mod_test_pred.loc[data_mod_test_pred["game_play_id"] == play].copy()
        num_frames_output = curr_play_info["num_frames_output"].unique()[0] #number of frames to predict
        player_ids = curr_play_info["game_player_play_id"].unique() #players in this play
        last_frame_id = curr_play_info["frame_id"].unique()[0] #last pre-throw frame
        
        #loop over future frames
        for output_frame_id in range(1, num_frames_output + 1): 
            frame = last_frame_id + output_frame_id #current frame
    
            #info for all players in current frame
            #update with predicted x,y,dir,s,a
            curr_frame_all_players = curr_play_info.copy()
            #update frame timing stuff
            curr_frame_all_players["frame_id"] = frame
            curr_frame_all_players["prop_play_complete"] = (frame / (last_frame_id + num_frames_output))
            curr_frame_all_players["time_until_play_complete"] = ((last_frame_id + num_frames_output) - frame)*0.1
            curr_frame_all_players["time_elapsed_post_throw"] = (output_frame_id - 1)*0.1
            curr_frame_all_players["time_elapsed"] = frame*0.1
            
            #update throw to be post
            if output_frame_id > 1:
                curr_frame_all_players["throw"] = "post"
    
            #if frame is pre throw we already know the features and everything to predict x,y,dir,s,a
    
            #if frame is post throw then we need to update position and dir, s, a based on previous frame's predictions
            if curr_frame_all_players["throw"].unique()[0] == "post":
    
                #previous frame predictions
                prev_frame_all_players = result.copy()
    
                #update current x, y dir, s, a, as previous prediction
                curr_frame_all_players["x"] = prev_frame_all_players["pred_x"].values
                curr_frame_all_players["y"] = prev_frame_all_players["pred_y"].values
                curr_frame_all_players["est_dir"] = prev_frame_all_players["pred_dir"].values
                curr_frame_all_players["est_speed"] = prev_frame_all_players["pred_s"].values
                curr_frame_all_players["est_acc"] = prev_frame_all_players["pred_a"].values
    
                #initialize prediction cols to NaN
                for col in ["pred_x", "pred_y", "pred_dir", "pred_s", "pred_a"]:
                    curr_frame_all_players[col] = np.nan
    
                #derive features
    
                #closest features
                closest_player_features = get_closest_player_min_dist_dir(curr_frame_all_players)
                curr_frame_all_players = curr_frame_all_players.drop(columns=["closest_teammate_dist", "closest_teammate_dir_diff_position", "closest_teammate_dir_diff_heading",
                                                                              "closest_opponent_dist", "closest_opponent_dir_diff_position", "closest_opponent_dir_diff_heading",
                                                                              "closest_teammate", "closest_opponent", "closest_opponent_speed", "closest_opponent_acc", 
                                                                              "closest_teammate_speed", "closest_teammate_acc"])
                curr_frame_all_players = curr_frame_all_players.merge(closest_player_features, on="game_player_play_id", how="outer")
    
                #combine prev + curr frame to compute lag features
                prev_curr_frame_df = pd.concat(
                    [prev_frame_all_players, curr_frame_all_players], ignore_index=True
                ).sort_values(["game_player_play_id", "frame_id"])
    
                #derive features - group by game_player_play_id here
    
                with np.errstate(invalid="ignore"):
                    curr_frame_all_players = (prev_curr_frame_df
                                              .groupby("game_player_play_id")
                                              .apply(change_in_kinematics)
                                              .reset_index(drop=True))
                    curr_frame_all_players = derived_features(curr_frame_all_players)
    
                # keep only the current frame
                curr_frame_all_players = curr_frame_all_players.loc[
                    curr_frame_all_players["frame_id"] == frame
                ].reset_index(drop=True)
    
            #predict next x,y
            curr_frame_all_players["pred_dist_diff"] = (
                curr_frame_all_players["est_speed"]*0.1 +
                curr_frame_all_players["est_acc"]*0.5*(0.1**2)
            )
    
            angle_rad = np.deg2rad((90 - curr_frame_all_players["est_dir"]) % 360)
            curr_frame_all_players["pred_x"] = (
                curr_frame_all_players["x"] + np.cos(angle_rad) * curr_frame_all_players["pred_dist_diff"]
            )
            curr_frame_all_players["pred_y"] = (
                curr_frame_all_players["y"] + np.sin(angle_rad) * curr_frame_all_players["pred_dist_diff"]
            )
            
            curr_frame_all_players = curr_frame_all_players.drop(columns=["pred_dist_diff"])
    
            #predict dir, s, a
            #need to loop through each player since catboost load pool thing
            curr_frame_all_players_pred = []
            for player in player_ids:
    
                player_row = curr_frame_all_players.loc[
                    curr_frame_all_players["game_player_play_id"] == player
                ].copy()
    
                #set off/def models and pools
                if player_row["player_side"].unique() == "Offense":
                    dir_cat = dir_o
                    speed_cat = speed_o
                    acc_cat = acc_o
    
                    #pool
                    dir_pool = Pool(player_row[dir_o_features], cat_features = list(set(categorical_features) & set(dir_o_features)))
                    speed_pool = Pool(player_row[speed_o_features], cat_features = list(set(categorical_features) & set(speed_o_features)))
                    acc_pool = Pool(player_row[acc_o_features], cat_features = list(set(categorical_features) & set(acc_o_features)))
                else:
                    dir_cat = dir_d
                    speed_cat = speed_d
                    acc_cat = acc_d
                    
                    #pool
                    dir_pool = Pool(player_row[dir_d_features], cat_features = list(set(categorical_features) & set(dir_d_features)))
                    speed_pool = Pool(player_row[speed_d_features], cat_features = list(set(categorical_features) & set(speed_d_features)))
                    acc_pool = Pool(player_row[acc_d_features], cat_features = list(set(categorical_features) & set(acc_d_features)))
    
                #predict
                player_row["pred_dir"] = (player_row["est_dir"] + dir_cat.predict(dir_pool)[0])
                player_row["pred_s"] = np.exp(speed_cat.predict(speed_pool)[0]) 
                player_row["pred_a"] = acc_cat.predict(acc_pool)[0]
    
                curr_frame_all_players_pred.append(player_row)
    
            #store result for next iteration
            result = pd.concat(curr_frame_all_players_pred)
    
            # if play == 0: print(result[result["nfl_id"] == 53472][["frame_id", "player_height", "player_weight",
            #                                                        "x", "y", "est_dir", "est_speed", "est_acc", "num_frames_output", "prop_play_complete",
            #                                                       "throw", "time_elapsed_post_throw", "prev_dir_diff", "prev_a_diff", "ball_land_dir_diff",
            #                                                       "dist_ball_land", "time_elapsed", "time_until_play_complete", "out_bounds_dist", "out_bounds_dir_diff",
            #                                                       "closest_teammate_dist", "closest_teammate_dir_diff", "closest_opponent_dist", "closest_opponent_dir_diff",
            #                                                       "pred_x", "pred_y", "pred_dir", "pred_s", "pred_a"]])
    
            # if play == 0: 
            #     print(result[result["nfl_id"] == 53472][acc_d_features])
            #     row_1_acc_d_features = result[result["nfl_id"] == 53472][acc_d_features]
            
            #store
            final_predictions = pd.concat([final_predictions, result], ignore_index=True)
        #end frame loop
    #end play loop
    
    #join predictions on test
    test_key = test[["game_id", "nfl_id", "play_id", "frame_id"]]
    #convert frames starting at 1 post throw
    final_predictions["frame_id"] = final_predictions["frame_id"] - final_predictions["max_frame_pre_throw"]
    predictions = (test_key.merge(final_predictions, on=["game_id", "nfl_id", "play_id", "frame_id"], how="left"))
    
    #return only pred x,y
    predictions = predictions[["pred_x", "pred_y"]].rename(columns={"pred_x": "x", "pred_y": "y"})
    

    assert isinstance(predictions, (pd.DataFrame, pl.DataFrame))
    assert len(predictions) == len(test)
    return predictions

# When your notebook is run on the hidden test set, inference_server.serve must be called within 10 minutes of the notebook starting
# or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very
# first `predict` call, which does not have the usual 5 minute response deadline.
inference_server = kaggle_evaluation.nfl_inference_server.NFLInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/nfl-big-data-bowl-2026-prediction/',))

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


0


/tmp/ipykernel_13/1127190484.py:253: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_predictions = pd.concat([final_predictions, result], ignore_index=True)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/tmp/ipykernel_13/1127190484.py:253: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_predictions = pd.concat([final_predictions, result], ignore_index=True)


0


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/tmp/ipykernel_13/1127190484.py:253: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_predictions = pd.concat([final_predictions, result], ignore_index=True)


...

0


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/tmp/ipykernel_13/1127190484.py:253: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_predictions = pd.concat([final_predictions, result], ignore_index=True)


0


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/tmp/ipykernel_13/1127190484.py:253: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_predictions = pd.concat([final_predictions, result], ignore_index=True)


0
